# Assemble the Dataset for Sentiment Classifier

In [1]:
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import joblib

# 1.0 Prepare CCD_labeled

### 1.1 CCD_labeled train and validate

In [2]:
ccd_train_validate = pd.read_parquet(r"input_data\ccd_labeled_train_validate.parquet")
print(ccd_train_validate.shape)
display(ccd_train_validate.head())     

(13268, 6)


date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   
3  2024-10-05  2024-10-07_1017--3   
4  2024-10-05  2024-10-07_1017--4   

                                                text product   action  \
0                      i am still waiting on my card    card  inquire   
1  what can i do if my card still hasnt arrived a...    card  inquire   
2  i have been waiting over a week is the card st...    card  inquire   
3  can i track my card while it is in the process...    card  inquire   
4  how do i know if i will get my card or if it i...    card  inquire   

  sentiment  
0  negative  
1  negative  
2  negative  
3   neutral  
4   neutral

### 1.2 CCD_labeled test

In [3]:
ccd_test = pd.read_parquet(r"input_data\ccd_labeled_test.parquet")
print(ccd_test.shape)
display(ccd_test.head())  

(706, 6)


date                dlg_id  \
0  2024-10-05   2024-10-14__dlg_103   
1  2024-10-05   2024-10-14__dlg_103   
2  2024-10-05   2024-10-14__dlg_103   
3  2024-10-05  2024-10-14__dlg_1134   
4  2024-10-05  2024-10-14__dlg_1134   

                                                text  product    action  \
0  Hi, I’m really frustrated with my current expe...  account  transfer   
1  Honestly, I've been facing a lot of issues wit...  account  transfer   
2  I appreciate the offer, but this has been an o...  account  transfer   
3  Hi, Jordan! I'm Alice. I recently received my ...  account    report   
4  Certainly! There’s a transaction dated Septemb...  account    report   

  sentiment  
0  negative  
1  negative  
2  negative  
3  positive  
4  positive

### 1.3 Combine CCD_labeled Full

In [4]:
ccd_full = pd.concat([ccd_train_validate, ccd_test])
ccd_full.reset_index(drop=True, inplace=True)
print(ccd_full.shape)
display(ccd_full.head())
display(ccd_full.tail())
del ccd_train_validate
del ccd_test

(13974, 6)


date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   
3  2024-10-05  2024-10-07_1017--3   
4  2024-10-05  2024-10-07_1017--4   

                                                text product   action  \
0                      i am still waiting on my card    card  inquire   
1  what can i do if my card still hasnt arrived a...    card  inquire   
2  i have been waiting over a week is the card st...    card  inquire   
3  can i track my card while it is in the process...    card  inquire   
4  how do i know if i will get my card or if it i...    card  inquire   

  sentiment  
0  negative  
1  negative  
2  negative  
3   neutral  
4   neutral

date               dlg_id  \
13969  2024-10-05  2024-10-14__dlg_878   
13970  2024-10-05  2024-10-14__dlg_878   
13971  2024-10-05   2024-10-14__dlg_96   
13972  2024-10-05   2024-10-14__dlg_96   
13973  2024-10-05   2024-10-14__dlg_96   

                                                    text  product action  \
13969  It says something about a “server connection t...  website    pay   
13970  I didn’t think of using the mobile app. I’ll g...  website    pay   
13971  Hi, I'm trying to visit your bank's website, b...  website   find   
13972  I was trying something similar, but it seems l...  website   find   
13973  I just wanted to check my recent transactions ...  website   find   

      sentiment  
13969  negative  
13970  negative  
13971   neutral  
13972   neutral  
13973   neutral

In [5]:
ccd_full.columns

Index(['date', 'dlg_id', 'text', 'product', 'action', 'sentiment'], dtype='object')

In [6]:
ccd_full = ccd_full[['text', 'sentiment']].copy()
ccd_full.rename(columns={'sentiment': 'label_name'}, inplace=True)
ccd_full.reset_index(drop=True, inplace=True)
display(ccd_full.head(3))
print()

text label_name
0                      i am still waiting on my card   negative
1  what can i do if my card still hasnt arrived a...   negative
2  i have been waiting over a week is the card st...   negative

# 2.0 Load SST-5

In [7]:
ds_sst5 = load_dataset("SetFit/sst5")
ds_sst5

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2210
    })
})

In [8]:
df_sst5_train = pd.DataFrame(ds_sst5['train'])
print(df_sst5_train.shape)
display(df_sst5_train.head(3))

(8544, 3)


text  label     label_text
0  a stirring , funny and finally transporting re...      4  very positive
1  apparently reassembled from the cutting-room f...      1       negative
2  they presume their audience wo n't sit still f...      1       negative

In [9]:
df_sst5_validation = pd.DataFrame(ds_sst5['validation'])
print(df_sst5_validation.shape)
display(df_sst5_validation.head(3))

(1101, 3)


text  label label_text
0  in his first stab at the form , jacquot takes ...      2    neutral
1                       one long string of cliches .      1   negative
2  if you 've ever entertained the notion of doin...      1   negative

In [10]:
df_sst5_test = pd.DataFrame(ds_sst5['test'])
print(df_sst5_test.shape)
display(df_sst5_test.head(3))

(2210, 3)


text  label     label_text
0     no movement , no yuks , not much of anything .      1       negative
1  a gob of drivel so sickly sweet , even the eag...      0  very negative
2  ` how many more voyages can this limping but d...      2        neutral

<b>df_sst5<b>

In [11]:
df_sst5 = pd.concat([df_sst5_train, df_sst5_validation, df_sst5_test])
df_sst5.rename(columns={'label_text': 'label_name'}, inplace=True)
df_sst5.reset_index(drop=True, inplace=True)
print(df_sst5.shape)
display(df_sst5.head(3))
del df_sst5_train
del df_sst5_validation
del df_sst5_test

(11855, 3)


text  label     label_name
0  a stirring , funny and finally transporting re...      4  very positive
1  apparently reassembled from the cutting-room f...      1       negative
2  they presume their audience wo n't sit still f...      1       negative

# 3.0 Combined Full Dataset

### 3.1 Check that label_text and labels are same for both `ccd_full` and `df_sst5` datasets

In [12]:
lbl2id = {
    "very negative": 0,
    "negative": 1,
    "neutral": 2,
    "positive": 3,
    "very positive": 4
}
id2lbl = {
    0: "very negative",
    1: "negative",
    2: "neutral",
    3: "positive",
    4: "very positive"
}

In [13]:
ccd_full['label'] = ccd_full['label_name'].apply(lambda x: lbl2id[x])
ccd_full = ccd_full[['text', 'label', 'label_name']]
print(ccd_full.shape)
display(ccd_full.sample(5))

(13974, 3)


text  label label_name
5521                  should i reinstall the payment app      2    neutral
6814                  show me my current account balance      2    neutral
4474   please help me figure out the reason for the o...      2    neutral
10656  I don’t have my account number handy because I...      1   negative
1104                can i get an account for my daughter      2    neutral

In [14]:
df_sst5['label'] = df_sst5['label_name'].apply(lambda x: lbl2id[x])
df_sst5 = df_sst5[['text', 'label', 'label_name']]
print(df_sst5.shape)
display(df_sst5.sample(5))

(11855, 3)


text  label     label_name
4096  me without you has a bracing truth that 's ref...      4  very positive
4237  it looks closely , insightfully at fragile , c...      3       positive
2515  however sincere it may be , the rising place n...      1       negative
7747  a film with almost as many delights for adults...      4  very positive
1795  -lrb- seems -rrb- even more uselessly redundan...      1       negative

In [15]:
df_sst5['label'].unique(), df_sst5['label_name'].unique() 
# note, the order might not be left to right, not depend on it
# will need to remap it just in case

(array([4, 1, 2, 3, 0], dtype=int64),
 array(['very positive', 'negative', 'neutral', 'positive',
        'very negative'], dtype=object))

### 3.2 Combine `ccd_full` and `df_sst5`

In [16]:
ccd_full.shape, df_sst5.shape

((13974, 3), (11855, 3))

In [18]:
ccd_full = pd.concat([ccd_full, df_sst5])
ccd_full.reset_index(drop=True, inplace=True)
print(ccd_full.shape)
display(ccd_full.head())
display(ccd_full.tail())

(25829, 3)


text  label label_name
0                      i am still waiting on my card      1   negative
1  what can i do if my card still hasnt arrived a...      1   negative
2  i have been waiting over a week is the card st...      1   negative
3  can i track my card while it is in the process...      2    neutral
4  how do i know if i will get my card or if it i...      2    neutral

text  label     label_name
25824  the problem with concept films is that if the ...      1       negative
25825  safe conduct , however ambitious and well-inte...      1       negative
25826  a film made with as little wit , interest , an...      1       negative
25827  to enjoy this movie 's sharp dialogue and deli...      2        neutral
25828  but here 's the real damn : it is n't funny , ...      0  very negative

# 3.3 Save to Disk

In [20]:
ccd_full.to_parquet("output_data/ccd_full.parquet", index=False)
ccd_full.to_csv("output_data/ccd_full.csv", index=False, encoding='utf-8-sig')